In [10]:
import tensorflow as tf
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Activation,Dropout

In [11]:
data_dir=r'../input/gtsrb-german-traffic-sign'
vsplit=.2 
IMAGE_SIZE=[224, 224]
IMAGE_SHAPE=[224,224,3]
train_model=ImageDataGenerator(rescale=1./255,shear_range=0.2,zoom_range = 0.2,
                                   horizontal_flip=True,validation_split=vsplit)
test_model=ImageDataGenerator(rescale=1./255,validation_split=vsplit)
training_data=train_model.flow_from_directory('../input/gtsrb-german-traffic-sign/Train',
                                                 target_size=IMAGE_SIZE,batch_size=32,
                                                 class_mode='categorical',subset='training',
                                                 shuffle=True,seed=123)
testing_data=test_model.flow_from_directory('../input/gtsrb-german-traffic-sign/Test', 
                                                target_size=IMAGE_SIZE,batch_size = 32,
                                                class_mode='categorical',subset='validation',
                                                shuffle=True,seed=123)
class_dict=training_data.class_indices
classes=list(class_dict.keys())
print('LIST OF CLASSES',classes)
print('CLASS DICTIONARY',class_dict)
number_of_classes=len(classes)
print('Number of classes=',number_of_classes)

Found 31368 images belonging to 43 classes.
Found 0 images belonging to 0 classes.
LIST OF CLASSES ['0', '1', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '2', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '3', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '4', '40', '41', '42', '5', '6', '7', '8', '9']
CLASS DICTIONARY {'0': 0, '1': 1, '10': 2, '11': 3, '12': 4, '13': 5, '14': 6, '15': 7, '16': 8, '17': 9, '18': 10, '19': 11, '2': 12, '20': 13, '21': 14, '22': 15, '23': 16, '24': 17, '25': 18, '26': 19, '27': 20, '28': 21, '29': 22, '3': 23, '30': 24, '31': 25, '32': 26, '33': 27, '34': 28, '35': 29, '36': 30, '37': 31, '38': 32, '39': 33, '4': 34, '40': 35, '41': 36, '42': 37, '5': 38, '6': 39, '7': 40, '8': 41, '9': 42}
Number of classes= 43


In [3]:
base_model_vgg=tf.keras.applications.VGG19(include_top=False, weights="imagenet",input_shape=IMAGE_SHAPE, pooling='max')

2022-09-02 04:02:53.795003: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-02 04:02:53.921135: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-02 04:02:53.921874: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-02 04:02:53.923405: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

80150528/80134624 [==============================] - 3s 0us/step


In [4]:
x_vgg=base_model_vgg.output
base_model_vgg.trainable=False

In [5]:
x_vgg=Dense(256,activation='relu')(x_vgg)
x_vgg=Dropout(rate=.45,seed=123)(x_vgg)        
output_vgg=Dense(number_of_classes, activation='softmax')(x_vgg)
model_vgg=Model(inputs=base_model_vgg.input,outputs=output_vgg)
model_vgg.compile(Adam(learning_rate=.001),loss='categorical_crossentropy',metrics=['accuracy'])

In [6]:
epochs=5
rlronp_vgg=tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss",factor=0.5,patience=1,verbose=1)
estop_vgg=tf.keras.callbacks.EarlyStopping(monitor="val_loss",patience=3,verbose=1,restore_best_weights=True)
history_vgg=model_vgg.fit(x=training_data,epochs=epochs,verbose=1,validation_data=testing_data,
                  callbacks=[rlronp_vgg,estop_vgg],validation_steps=None,shuffle=True,initial_epoch=0)

2022-09-02 04:03:22.835223: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/5


2022-09-02 04:03:25.231900: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


981/981 [==============================] - 522s 522ms/step - loss: 2.6604 - accuracy: 0.2608
Epoch 2/5
981/981 [==============================] - 365s 372ms/step - loss: 2.0848 - accuracy: 0.3893
Epoch 3/5
981/981 [==============================] - 358s 365ms/step - loss: 1.8755 - accuracy: 0.4396
Epoch 4/5
981/981 [==============================] - 358s 365ms/step - loss: 1.7364 - accuracy: 0.4805
Epoch 5/5
981/981 [==============================] - 364s 371ms/step - loss: 1.6602 - accuracy: 0.4951


In [12]:
!pip install efficientnet

In [13]:
import efficientnet.tfkeras as efn
base_model_efn=efn.EfficientNetB0(include_top=False,weights="imagenet",input_shape=IMAGE_SHAPE, pooling='max')

In [14]:
x_efn=base_model_efn.output
base_model_efn.trainable=False

In [15]:
x_efn=Dense(256,activation='relu')(x_efn)
x_efn=Dropout(rate=.45,seed=123)(x_efn)   
output_efn=Dense(number_of_classes,activation='softmax')(x_efn)
model_efn=Model(inputs=base_model_efn.input,outputs=output_efn)
model_efn.compile(Adam(learning_rate=.001),loss='categorical_crossentropy',metrics=['accuracy'])

In [16]:
epochs=5
rlronp_efn=tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss",factor=0.5,patience=1,verbose=1)
estop_efn=tf.keras.callbacks.EarlyStopping( monitor="val_loss",patience=3,verbose=1,restore_best_weights=True)
history_efn=model_efn.fit(x=training_data,epochs=epochs,verbose=1,validation_data=testing_data,
                  callbacks=[rlronp_efn,estop_efn],validation_steps=None,shuffle=True,initial_epoch=0)

Epoch 1/5
981/981 [==============================] - 388s 390ms/step - loss: 2.3089 - accuracy: 0.3407
Epoch 2/5
981/981 [==============================] - 383s 390ms/step - loss: 1.7215 - accuracy: 0.4667
Epoch 3/5
981/981 [==============================] - 373s 381ms/step - loss: 1.5524 - accuracy: 0.5168
Epoch 4/5
981/981 [==============================] - 376s 384ms/step - loss: 1.4802 - accuracy: 0.5351
Epoch 5/5
981/981 [==============================] - 382s 390ms/step - loss: 1.3975 - accuracy: 0.5576
